<a href="https://colab.research.google.com/github/ckgpeace/EIP4/blob/master/Assignment5/Assignment5_PersonAttrubutes_vanilla1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!kill -9 -1

In [0]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 

In [0]:
%tensorflow_version 1.x

import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from keras.layers import Activation, Flatten, Dense, Dropout, GlobalAveragePooling2D, AveragePooling2D
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D, SeparableConv2D
from keras.applications import VGG16
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input, BatchNormalization
from keras.models import Model
from keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator

In [0]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

In [0]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

In [0]:
import keras
import numpy as np

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, batch_size=32, shuffle=True, augmentation = None):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        self.augmentation = augmentation

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        image = np.stack([cv2.resize(cv2.imread(item["image_path"]), (112,112)) for _, item in items.iterrows()])
        #Image Normalization
        if self.augmentation is not None:
          image = self.augmentation.flow(image,shuffle=False, batch_size = 32 ).next()
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values
        }
        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

In [0]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.15, random_state = 404)
train_df.shape, val_df.shape

In [0]:
train_df.head()

In [0]:
# create train and validation data generators
train_gen = PersonDataGenerator(train_df, batch_size=32, shuffle=True, 
                                augmentation = ImageDataGenerator(horizontal_flip=True, 
                                                                  width_shift_range=0.2,
                                                                  height_shift_range=0.2,
                                                                  rotation_range=0,
                                                                  zoom_range=0.2,
                                                                  rescale=1./255))
valid_gen = PersonDataGenerator(val_df, batch_size=32, shuffle=False, augmentation= ImageDataGenerator(rescale=1./255))

In [0]:
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

In [0]:

inp = Input(shape = (112,112,3))

### block 1
x = SeparableConv2D(filters=32, kernel_size=(3, 3), padding='valid')(inp) 
x = Activation('relu')(x)
x = BatchNormalization()(x)

x = SeparableConv2D(filters=32, kernel_size=(3, 3), padding='valid')(x) 
x = Activation('relu')(x)
x = BatchNormalization()(x)

### block 2
x = SeparableConv2D(filters=64, kernel_size=(3, 3), padding='valid')(inp) 
x = Activation('relu')(x)
x = BatchNormalization()(x)

x = SeparableConv2D(filters=64, kernel_size=(3, 3), padding='valid')(inp) 
x = Activation('relu')(x)
x = BatchNormalization()(x)

# Pooling 
x = AveragePooling2D()(x)

# ================================================================

### block 2 
x = SeparableConv2D(filters=128, kernel_size=(3, 3), padding='valid')(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)

x = SeparableConv2D(filters=128, kernel_size=(3, 3), padding='valid')(x)
x = Activation('relu')(x)
x = Dropout(0.05)(x)
x = BatchNormalization()(x)

# Pooling 
x = AveragePooling2D()(x)

# ================================================================

### block 3
x = SeparableConv2D(filters=256, kernel_size=(3, 3), padding='valid')(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)

x = SeparableConv2D(filters=256, kernel_size=(3, 3), padding='valid')(x)
x = Activation('relu')(x)
x = Dropout(0.05)(x)
x = BatchNormalization()(x)

# Pooling 
x = AveragePooling2D()(x)

# ================================================================
# block  - Last CNN

x = SeparableConv2D(filters=512, kernel_size=(3, 3), padding='valid')(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)

x = SeparableConv2D(filters=512, kernel_size=(3, 3), padding='valid')(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)

# Adding Dense layer
def final(in_layer, num_units, class_name, output_name):
  # Conv with class size
  x = SeparableConv2D(filters=num_units[class_name], kernel_size=(1, 1), padding='valid')(in_layer)
  # GAP
  x = GlobalAveragePooling2D()(x)
  x = Activation('softmax', name = output_name)(x)
  return x

gender = final(in_layer = x, num_units = num_units,  class_name = "gender", output_name = "gender_output")
image_quality = final(in_layer = x, num_units = num_units,  class_name = "image_quality", output_name = "image_quality_output")
age = final(in_layer = x, num_units = num_units,  class_name = "age", output_name = "age_output")
weight = final(in_layer = x, num_units = num_units,  class_name = "weight", output_name = "weight_output")
bag = final(in_layer = x, num_units = num_units,  class_name = "bag", output_name = "bag_output")
footwear = final(in_layer = x, num_units = num_units,  class_name = "footwear", output_name = "footwear_output")
emotion = final(in_layer = x, num_units = num_units,  class_name = "emotion", output_name = "emotion_output")
pose = final(in_layer = x, num_units = num_units,  class_name = "pose", output_name = "pose_output")

model = Model(inputs = inp,outputs=[gender, image_quality, age, weight, bag, pose, footwear, emotion])
model.summary()

In [0]:
# from keras.utils import plot_model
# plot_model(model)
import time, psutil
uptime = time.time() - psutil.boot_time()
remain = 12*60*60 - uptime
remain/(60*60)

In [0]:
losses = {
	"gender_output": "categorical_crossentropy",
	"image_quality_output": "categorical_crossentropy",
	"age_output": "categorical_crossentropy",
	"weight_output": "categorical_crossentropy",
  "bag_output":  "categorical_crossentropy",
  "pose_output": "categorical_crossentropy",
  "footwear_output": "categorical_crossentropy",
  "emotion_output": "categorical_crossentropy"
}

loss_weights = {"gender_output": 1.0, "image_quality_output": 1.0, "age_output": 1.0, "weight_output" :1.0,  "bag_output": 1.0, "pose_output": 1.0,  "footwear_output": 1.0, "emotion_output": 1.0 }



from keras.callbacks import LearningRateScheduler, ModelCheckpoint, EarlyStopping

decay_factor = 0.01
def scheduler(epoch, lr):
  return round(lr * 1/(1 + decay_factor * epoch), 10)

opt = SGD(lr=0.01, momentum=0.9)

model.compile( optimizer=opt, loss = losses, loss_weights=loss_weights, metrics=["accuracy"])



In [0]:
# model_path = '/content/gdrive/VGG16_vanila_1.h5'
# checkpoint = ModelCheckpoint(model_path, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='max')

# Checkpoint saving
save_dir = os.path.join(os.getcwd(), "saved_models")
model_name = "model.{epoch:03d}.h5"
if not os.path.isdir(save_dir):
  os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

checkpoint = ModelCheckpoint(model_path, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False)

early = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10, verbose=1, mode='auto')

model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=100,
    verbose=1,
    callbacks=[checkpoint, early, LearningRateScheduler(scheduler, verbose=1)]
    # callbacks=[checkpoint, early, LearningRateScheduler(scheduler, verbose=1),TensorBoardColabCallback(tbc)]
)

In [0]:
def evaluate_model(model):
  results = model.evaluate_generator(valid_gen, verbose=1)
  accuracies = {}
  losses = {}
  for k, v in zip(model.metrics_names, results):
    if k.endswith('acc'):
      accuracies[k] = round(v * 100, 4)
    else:
      losses[k] = v
  return accuracies

evaluate_model(model)

# results = model.evaluate_generator(valid_gen, verbose =1)
# dict(zip(model.metrics_names, results))


In [0]:
import matplotlib.pyplot as plt
def plot_model_history(model_history):
    fig, axs = plt.subplots(1,1,figsize=(15,5))
    # summarize history for accuracy
    # axs[0].plot(range(1,len(model_history.history.history['acc'])+1),model_history.history['acc'])
    # axs[0].plot(range(1,len(model_history.history['val_acc'])+1),model_history.history['val_acc'])
    # axs[0].set_title('Model Accuracy')
    # axs[0].set_ylabel('Accuracy')
    # axs[0].set_xlabel('Epoch')
    # axs[0].set_xticks(np.arange(1,len(model_history.history['acc'])+1),len(model_history.history['acc'])/10)
    # axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs.plot(range(1,len(model_history.history.history['loss'])+1),model_history.history.history['loss'])
    axs.plot(range(1,len(model_history.history.history['val_loss'])+1),model_history.history.history['val_loss'])
    axs.set_title('Model Loss')
    axs.set_ylabel('Loss')
    axs.set_xlabel('Epoch')
    axs.set_xticks(np.arange(1,len(model_history.history.history['loss'])+1),len(model_history.history.history['loss'])/10)
    axs.legend(['train', 'val'], loc='best')
    plt.show()
# plot model history
plot_model_history(model)